In [2]:
library("INLA")
library("dplyr")
library("tidyverse")

In [3]:
data = read.csv("JHU_rollup_as_of_2020-10-01.csv", header = TRUE)
county = read.csv("county_pop.csv",header = TRUE)
data = merge(data,county, by="fips")

lag.data = data %>% group_by(fips) %>%arrange(date)%>% mutate(daily = confirmed - lag(confirmed, 1))

lag.data %>% mutate(daily2 = ifelse(is.na(daily),0, daily)) ->lag.noNA #daily2 NA values replaced with 0 

lag.noNA %>% mutate(daily2 = ifelse(daily < 0, 0, daily)) ->lag.clean  #negative daily2 values replaced with 0

#see how many lags are negative
pos<-nrow(lag.data[lag.data$daily>=0,])
neg<-nrow(lag.data[lag.data$daily<0,])
percent.neg<- neg/(neg+pos) #Note: Only 2% of the daily observations are negative. We will replace these with zero

In [4]:
head(lag.clean)

fips,date,county_name,province_state,country_region,confirmed,deaths,recovered,active,population,daily,daily2
<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>
10001,2020-03-22,Kent,Delaware,US,5,0,0,0,178550,NA,NA
10003,2020-03-22,New Castle,Delaware,US,39,0,0,0,559335,NA,NA
10005,2020-03-22,Sussex,Delaware,US,12,0,0,0,229286,NA,NA
11001,2020-03-22,District of Columbia,District of Columbia,US,102,2,0,0,702455,NA,NA
11001,2020-03-22,District of Columbia,District of Columbia,US,102,0,0,0,702455,0,0
12001,2020-03-22,Alachua,Florida,US,35,0,0,0,269956,NA,NA
